In [2]:
import pandas as pd
import wandb
from pathlib import Path
import json
import numpy as np

In [3]:
method="procrustes" 
extend_exp="image_disp"
vm_name = {
    'resnet101': 'ResNet101',
    'resnet152': 'ResNet152',
    'resnet18': 'ResNet18',
    'resnet34': 'ResNet34',
    'resnet50': 'ResNet50',
    'segformer-b0-finetuned-ade-512-512': 'SegFormer<sub>B0</sub>',
    'segformer-b1-finetuned-ade-512-512': 'SegFormer<sub>B1</sub>',
    'segformer-b2-finetuned-ade-512-512': 'SegFormer<sub>B2</sub>',
    'segformer-b3-finetuned-ade-512-512': 'SegFormer<sub>B3</sub>',
    'segformer-b4-finetuned-ade-512-512': 'SegFormer<sub>B4</sub>',
    'segformer-b5-finetuned-ade-640-640': 'SegFormer<sub>B5</sub>',
    'vit-mae-base': 'MAE<sub>Base</sub>',
    'vit-mae-huge': 'MAE<sub>Huge</sub>',
    'vit-mae-large': 'MAE<sub>Large</sub>'
}

model_name = {'bert_uncased_L-2_H-128_A-2': "BERT<sub>TINY</sub>",
                'bert_uncased_L-4_H-256_A-4': "BERT<sub>MINI</sub>" ,
                'bert_uncased_L-4_H-512_A-8': "BERT<sub>SMALL</sub>",
                'bert_uncased_L-8_H-512_A-8': "BERT<sub>MEDIUM</sub>",
                'bert-base-uncased': "BERT<sub>BASE</sub>",
                'bert-large-uncased': "BERT<sub>LARGE</sub>",
                'gpt2': "GPT2<sub>BASE</sub>",
                'gpt2-medium': "GPT2<sub>MEDIUM</sub>",
                'gpt2-large': "GPT2<sub>LARGE</sub>",
                'gpt2-xl': "GPT2<sub>XL</sub>",
                'opt-125m': "OPT<sub>125M</sub>",
                'opt-1.3b': "OPT<sub>1.3B</sub>",
                'opt-6.7b': "OPT<sub>6.7B</sub>",
                'opt-30b': "OPT<sub>30B</sub>",
                'opt-66b': "OPT<sub>66B</sub>",
                'Llama-2-7b-hf': "Llama-2<sub>7B</sub>",
                'Llama-2-13b-hf': "Llama-2<sub>13B</sub>",
                'Llama-2-70b-hf': "Llama-2<sub>70B</sub>",
                "fasttext":"fastText"}

model_size = {'bert_uncased_L-2_H-128_A-2':4.4, 'bert_uncased_L-4_H-256_A-4':11.3, 'bert_uncased_L-4_H-512_A-8':29.1, 
    'bert_uncased_L-8_H-512_A-8':41.7, 'bert-base-uncased':110, 'bert-large-uncased':340, 
    'gpt2':117,  'gpt2-large':762, 'gpt2-xl':1542, 'opt-125m':125, 'opt-6.7b':6700, 'opt-30b':30000,'opt-66b':66000,
    'Llama-2-7b-hf': 7000,
    'Llama-2-13b-hf': 13000,
    'Llama-2-70b-hf': 70000,}

model_tags = {
    "ft": "fasttext",
    "bert": "bert",
    "gpt2": "gpt2",
    "opt": "opt"
}

bins = {
    "_low":"low",
    "_medium":'medium',
    "_high":"high"
}

bins_order = {
    "_low":"0",
    "_medium":'1',
    "_high":"2"
}

set_name = "cleaned"

In [4]:


api = wandb.Api()
df = pd.DataFrame()

for model_alias in ["VM"]:
    # method_name = "Procrustes Analysis" if method == "procrustes" else "Ridge Regression"
    runs = api.runs(path=f"jiaangli00/image2{model_alias}-TACL-{set_name}{extend_exp}")
    # metrics = runs.summary["Results"]

    # data = []
    for i, single_run in enumerate(runs):
        metric = json.load(single_run.file(single_run.summary["Results"]["path"]).download(exist_ok=True))
        df = pd.concat([df,pd.DataFrame(metric["data"], columns=metric["columns"])])

# # df['Subjects'] = df['Subjects'].replace({f'brain_{i}': f'Subject-{i}' for i in range(1, num_subs + 1)})
# df['Layers'] = df['Layers'].replace({f'layer_{i}': f'layer-{i})
df['Models_size'] = df['LM'].copy()
df['Models_size'] = df['Models_size'].replace(model_size)
df['VM'] = df['VM'].replace(vm_name)
df['LM'] = df['LM'].replace(model_name)
df['Bins_order'] = df['Bins'].copy()
df['Bins_order'] = df['Bins_order'].replace(bins_order)
df["Bins"] = df["Bins"].replace(bins)
# #
group_names: list[str] = ["LM", "VM"]
if extend_exp != "":
    group_names.append("Bins")
    group_names.append("Bins_order")
#
precision_csls, precision_nn = [], []
for k in [100]:
    precision_csls.append(f'P@{k}-CSLS')
    precision_nn.append(f'P@{k}-NN')

precision = precision_csls
#
precision.append("Models_size")
# precision.append("Bins_order")

In [5]:
seed_avg = df.groupby(group_names)[precision].mean().round(1).reset_index()
_selected = seed_avg.loc[(seed_avg["LM"] == "BERT_LARGE") | (seed_avg["LM"] == "GPT2_XL") | (seed_avg["LM"] == "OPT_30B") | (seed_avg["LM"] == "Llama-2-13B")]
# _selected
# seed_avg.to_csv("lang_disp.csv")
seed_avg

,LM,VM,Bins,Bins_order,P@100-CSLS,Models_size
0,BERT<sub>BASE</sub>,MAE<sub>Huge</sub>,high,2,27.0,110.0
1,BERT<sub>BASE</sub>,MAE<sub>Huge</sub>,low,0,37.4,110.0
2,BERT<sub>BASE</sub>,MAE<sub>Huge</sub>,medium,1,36.5,110.0
3,BERT<sub>BASE</sub>,ResNet152,high,2,39.7,110.0
4,BERT<sub>BASE</sub>,ResNet152,low,0,48.1,110.0
...,...,...,...,...,...,...
121,OPT<sub>6.7B</sub>,ResNet152,low,0,67.5,6700.0
122,OPT<sub>6.7B</sub>,ResNet152,medium,1,61.0,6700.0
123,OPT<sub>6.7B</sub>,SegFormer<sub>B5</sub>,high,2,55.7,6700.0
124,OPT<sub>6.7B</sub>,SegFormer<sub>B5</sub>,low,0,58.8,6700.0


In [6]:
table = _selected.loc[(seed_avg["VM"] == "MAE-Huge") | (seed_avg["VM"] == "SegFormer-B5") | (seed_avg["VM"] == "ResNet152") ]

In [7]:
table

,LM,VM,Bins,Bins_order,P@100-CSLS,Models_size


In [8]:
# table2 = _selected.loc[(seed_avg["VM"] != "MAE-Huge") & (seed_avg["VM"] != "SegFormer-B5") & (seed_avg["VM"] != "ResNet152") ]
table2 = seed_avg.loc[(seed_avg["LM"] != "OPT<sub>30B</sub>") & (seed_avg["LM"] != "Llama-2<sub>13B</sub>") & (seed_avg["LM"] != "GPT2<sub>XL</sub>") & (seed_avg["LM"] != "BERT<sub>LARGE</sub>") ]
table2 = table2.sort_values(by=['Bins_order',"Models_size"], ascending=True)
table2

,LM,VM,Bins,Bins_order,P@100-CSLS,Models_size
46,BERT<sub>TINY</sub>,MAE<sub>Huge</sub>,low,0,14.9,4.4
49,BERT<sub>TINY</sub>,ResNet152,low,0,20.0,4.4
52,BERT<sub>TINY</sub>,SegFormer<sub>B5</sub>,low,0,14.9,4.4
28,BERT<sub>MINI</sub>,MAE<sub>Huge</sub>,low,0,21.5,11.3
31,BERT<sub>MINI</sub>,ResNet152,low,0,29.0,11.3
...,...,...,...,...,...,...
120,OPT<sub>6.7B</sub>,ResNet152,high,2,55.4,6700.0
123,OPT<sub>6.7B</sub>,SegFormer<sub>B5</sub>,high,2,55.7,6700.0
90,Llama-2<sub>7B</sub>,MAE<sub>Huge</sub>,high,2,34.7,7000.0
93,Llama-2<sub>7B</sub>,ResNet152,high,2,47.4,7000.0


In [9]:
import plotly.express as px
# import plotly.graph_objects as go

# fig = go.Figure()
traces = []
# df['VM_Bins'] = table2['VM'] + '_' + table2['Bins']
color_scale = px.colors.qualitative.Set3

# Create stacked and grouped bar chart
fig = px.bar(table2, x='LM', y='P@100-CSLS', color='Bins', facet_col="VM",
             labels={'P@100-CSLS': 'P@100'},
            #  facet_col_wrap=3,
            #  category_orders={'VM': ['MAE-Huge', 'ResNet152', 'SegFormer-B5']},
             color_discrete_map={'low': color_scale[5],"medium":color_scale[0],"high":color_scale[3]},
             height=500, width=1500,
             
            #  facet_col_titles='none'
            #  barmode="group"
             )


for i, trace in enumerate(fig.data):
    # trace.marker.color = color_scale[i%3]  # Set color
    trace.marker.opacity = 0.8  # Set opacity


# Update layout for better appearance
fig.update_layout(barmode='overlay', showlegend=True,
                  font=dict(size=30),
                  yaxis=dict(title=dict(text='P@100', font=dict(size=30)), tickfont=dict(size=25)),  # Change y-axis font size
                  legend=dict(title=dict(text='Img Disp.', font=dict(size=30))),
                  xaxis=dict(title=dict(text=''),tickfont=dict(size=20)),
                  xaxis2=dict(title=dict(text=''),tickfont=dict(size=20)),
                  xaxis3=dict(title=dict(text=''),tickfont=dict(size=20)),
                  template="plotly_white"
                  ) 

fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))

# fig.update_xaxes(title_text='', showgrid=False)

# fig.update_xaxes(title_font=dict(size=20), tickfont=dict(size=20))

# Show the plot
fig.show()
# fig.write_image(f"img_disp_remain_{set_name}.pdf")

In [10]:
import plotly.express as px
# import plotly.graph_objects as go

# fig = go.Figure()
traces = []
# df['VM_Bins'] = table2['VM'] + '_' + table2['Bins']
color_scale = px.colors.qualitative.Set3

# Create stacked and grouped bar chart
fig = px.scatter(table2, x='LM', y='P@100-CSLS', color='Bins', facet_col="VM",
             labels={'P@100-CSLS': 'P@100'},
             color_discrete_map={'low': color_scale[5],"medium":color_scale[0],"high":color_scale[3]},
             height=500, width=1500,
             symbol="Bins",
            #  radar_polar=True
            #  facet_col_titles='none'
            #  barmode="group"
             )


for i, trace in enumerate(fig.data):
    if "Bins=medium" in trace["hovertemplate"]:
    # trace.marker.color = color_scale[i%3]  # Set color
        trace.marker.opacity = 0.6  # Set opacity
    elif "Bins=high" in trace["hovertemplate"]:
        trace.marker.opacity = 0.5  # Set opacity
    else:
        trace.marker.opacity = 1
    # trace.marker.color = color_scale[i%3]  # Set color
    # trace.marker.opacity = 0.6  # Set opacity
    trace.marker.size=15


# Update layout for better appearance
fig.update_layout(barmode='group', showlegend=True,
                  font=dict(size=30),
                  yaxis=dict(title=dict(text='P@100', font=dict(size=30)), tickfont=dict(size=25)),  # Change y-axis font size
                  legend=dict(title=dict(text='Img Disp.', font=dict(size=30))),
                  xaxis=dict(title=dict(text=''),tickfont=dict(size=20)),
                  xaxis2=dict(title=dict(text=''),tickfont=dict(size=20)),
                  xaxis3=dict(title=dict(text=''),tickfont=dict(size=20)),
                  xaxis4=dict(title=dict(text=''),tickfont=dict(size=20)),
                  template="plotly_white"
                  ) 

fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))

# fig.update_xaxes(title_text='', showgrid=False)

# fig.update_xaxes(title_font=dict(size=20), tickfont=dict(size=20))

# Show the plot
fig.show()
fig.write_image(f"img_disp_remain_{set_name}_v1.pdf")

In [21]:
# import plotly.express as px
# import plotly.graph_objects as go

# # VMs = ["ResNet101",  "ResNet34", "SegFormer-B4", "SegFormer-B2", "SegFormer-B1" ]
# colors = px.colors.qualitative.Set2
# fig = go.Figure()
# classes = table2['LM'].unique()
# VMs = table2['VM'].unique()
# # print(classes)

# # Create radar chart traces for each VM
# traces = []
# for i, vm in enumerate(VMs):
#     trace = go.Scatterpolar(
#         r=table2[table2['VM'] == vm]["P@100-CSLS"].tolist(),
#         theta=classes,
#         fill='toself',
#         name=,
#         line=dict(color=colors[i]),
#         marker=dict(color=colors[i])
#     )
#     traces.append(trace)

# for trace in traces:
#     fig.add_trace(trace)

# # fig.add_trace(go.Scatterpolar(
# #       r=[1, 5, 2, 2, 3],
# #       theta=categories,
# #       fill='toself',
# #       name='Product A'
# # ))
# # fig.add_trace(go.Scatterpolar(
# #       r=[4, 3, 2.5, 1, 2],
# #       theta=categories,
# #       fill='toself',
# #       name='Product B'
# # ))

# fig.update_layout(
#   polar=dict(
#     angularaxis=dict(
#       visible=True,
#       gridwidth=3,
#       tickfont=dict(size=20)  # Adjust the font size here
#         ),
#     radialaxis=dict(
#       visible=True,
#       range=[0, 70],
#       tickfont=dict(size=20)
#       # line_close=True
#     )),
#   showlegend=True
# )

# fig.update_layout(
#     width=1200, 
#     height=1000, 
#     legend=dict(x=0, y=1.1, orientation="h",font=dict(size=20)), 
#     template="plotly",
#     font=dict(size=30)
#     )

# # fig.write_image(f"left_vm_vs_LMs_{set_name}_v3.pdf")


In [33]:
# table.to_csv("image_disp.csv")

In [37]:
for j in ["SegFormer-B5", "MAE-Huge", "ResNet152"]:
    print(j)
    for i in ["_low", "_medium", "_high"]:
        all_values = table.loc[(_selected["VM"] == j) & (_selected["Bins"] == i), precision]
        # print(all_values)
        formatted_values = " & ".join(f"{value}" for one_model_value in all_values.values for value in one_model_value)
        print(f"& {i[1:]} & " + formatted_values + "\\\\")

SegFormer-B5
& low & 43.2 & 47.8 & 58.7 & 60.6\\
& medium & 42.2 & 48.7 & 56.9 & 60.4\\
& high & 43.3 & 47.1 & 53.3 & 57.6\\
MAE-Huge
& low & 41.1 & 44.7 & 54.9 & 56.7\\
& medium & 37.6 & 43.6 & 53.8 & 55.3\\
& high & 32.7 & 40.1 & 46.7 & 54.3\\
ResNet152
& low & 50.8 & 57.6 & 71.3 & 70.5\\
& medium & 45.9 & 51.8 & 60.8 & 63.4\\
& high & 43.4 & 46.8 & 56.7 & 58.4\\


In [21]:
for j in ["BERT_LARGE", "GPT2_XL", "OPT_30B", "Llama-2-13B"]:
    print(j)
    for i in ["_low", "_medium", "_high"]:
        all_values = table.loc[(_selected["LM"] == j) & (_selected["Bins"] == i), precision].values
        formatted_values = " & ".join(f"{value}" for one_model_value in all_values for value in one_model_value)
        print(f"& {i[1:]} & " + formatted_values + "\\\\")


BERT_LARGE
& low & \\
& medium & \\
& high & \\
GPT2_XL
& low & 46.3 & 53.9 & 50.3\\
& medium & 42.0 & 52.3 & 47.0\\
& high & 25.2 & 34.3 & 30.7\\
OPT_30B
& low & 60.4 & 67.6 & 63.2\\
& medium & 55.1 & 65.1 & 59.7\\
& high & 35.6 & 43.7 & 40.4\\
Llama-2-13B
& low & 58.7 & 68.5 & 62.3\\
& medium & 52.9 & 65.1 & 57.3\\
& high & 32.3 & 42.1 & 36.8\\
